In [1]:
import matplotlib.pyplot as plt
import stompy.model.delft.dflow_model as dfm
import stompy.model.hydro_model as hm
import xarray as xr
import pandas as pd
%matplotlib notebook

In [5]:
import sfb_csc
Model=sfb_csc.SfbCsc

DFM_ROOT NOT SET


In [4]:
ls

barker_data.py             dem_cell_node_bathy.py     old/
batch_2d_2019_summer.sh    Dependencies.md            __pycache__/
batch_2d_2019_summer.sh~   Dependencies.md~           rough_regions.py
batch_3d_2019_summer.sh    extra_code.py              run_sfb_csc_test.py
batch_3d_2019_summer.sh~   geometry_input_fields.txt  sfb_csc.py
cache/                     hec_script.py              sfb_csc.py~
calibration.ipynb          local_config.py            slurm_out-61677139.output
data_2d_2019_summer/       local_config.py~           template.mdu
data_2d_2019_summer-v000/  local_config.py.in         Untitled.ipynb


In [2]:
runs=[
    dict(run_dir="data_2d_2019_summer",
         label="2D")
     ]

runs=pd.DataFrame(runs)

In [6]:
runs['model'] = runs['run_dir'].apply(lambda r: Model.load(r))

INFO:HydroModel:Duplicating depth to node_z_bed for less ambiguous naming, and assuming it was already positive-up


132935 nodes will get default of 0.02


In [9]:
# Dev for stage comparisons
his_ds=runs.loc[0,'model'].his_dataset()


In [10]:
his_ds # 51 stations.

<xarray.Dataset>
Dimensions:                                       (cross_section: 29, cross_section_geom_nNodes: 191, gategens: 1, laydim: 10, laydimw: 11, nFlowElemContourPts: 4, nFlowElemWithBnd: 10759, nFlowLink: 17115, nFlowLinkPts: 2, nNetLink: 19916, nNetLinkPts: 2, source_sink: 282, source_sink_geom_nNodes: 282, source_sink_pts: 1, station_geom_nNodes: 51, stations: 51, time: 2)
Coordinates:
    station_x_coordinate                          (stations) float64 ...
    station_y_coordinate                          (stations) float64 ...
    station_name                                  (stations) |S256 b'HS1                                                                                                                                                                                                                                                             ' ... b'mdot_HS2                                                                                                              

In [14]:
# Define observations. Use the same classes as for BCs
# since they already have the relevant dynamic loading
observations={
    'SRV':dict(stage=hm.NwisStageBC(11455420,name='SRV'),
               flow=hm.NwisFlowBC(11455420,name='SRV'))
}

In [ ]:
import stompy.model.data_comparison as dc

class StageCompare:
    pred=None # for stage, name of stations in DFM output
    obs=None  # will look for observations[obs]['stage']
    def __init__(self,**kw):
        utils.set_keywords(self,kw)
        
    def assemble_data(self):
        
    def figure(self):
        

In [11]:
# Start with Rio Vista
# HERE
comparisons=[
    # it should be relatively painless to compare two stations
    # from one model against another model or observation.
    StageCompare(runs=runs,pred='SRV',obs='SRV')
]

for comp in comparisons:
    comp.figure()

<xarray.DataArray 'stations' (stations: 51)>
array(['HS1', 'DOP', 'CCS', 'SG1', 'LN2', 'HAAS', 'SOI', 'SXS', 'RYI', 'HBW',
       'SSS', 'SUT', 'GES', 'DWS', 'LIB', 'LIS', 'FPX', 'SRV', 'LSHB', 'CCS1',
       'CCEH', 'BSPP', 'CCSC', 'WildlandsUpMarsh_stage', 'DLC', 'GSS', 'SDC',
       'SDI', 'TSL', 'CourtlandToe', 'LIY', 'ToeAtLiberty', 'HollandNBreach',
       'LibertyIslandCut', 'LibCutHolland', 'MIR', 'node320-drain',
       'node320-div', 'HST', 'DWSCTL', 'mdot_CC2', 'mdot_LN1', 'mdot_CA3',
       'mdot_HS1', 'mdot_BK1', 'mdot_CC1', 'mdot_BK2', 'mdot_CAUL', 'mdot_UL1',
       'mdot_CA1', 'mdot_HS2'], dtype='<U22')
Coordinates:
    station_x_coordinate  (stations) float64 ...
    station_y_coordinate  (stations) float64 ...
    station_name          (stations) |S256 b'HS1                                                                                                                                                                                                                      